# segment_card

> Segment card component with view and split modes

In [ ]:
#| default_exp components.segment_card

In [ ]:
#| export
from typing import Any, Callable, Set
import json

from fasthtml.common import Div, Span, Button, P

# DaisyUI components
from cjm_fasthtml_daisyui.components.actions.button import (
    btn, btn_sizes, btn_styles, btn_colors, btn_modifiers
)
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body
from cjm_fasthtml_daisyui.components.data_display.kbd import kbd, kbd_sizes
from cjm_fasthtml_daisyui.components.feedback.tooltip import tooltip, tooltip_placement
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, min_h
from cjm_fasthtml_tailwind.utilities.typography import (
    font_size, font_weight, font_family, leading, italic, uppercase, text_align
)
from cjm_fasthtml_tailwind.utilities.layout import position, top, left, right
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.utilities.effects import opacity
from cjm_fasthtml_tailwind.utilities.interactivity import cursor, select
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, flex_wrap, items, gap, grow, shrink
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Lucide icons
from cjm_fasthtml_lucide_icons.factory import lucide_icon

# Card stack library
from cjm_fasthtml_card_stack.core.constants import CardRole
from cjm_fasthtml_card_stack.core.models import CardRenderContext

# Token selector library
from cjm_fasthtml_token_selector.components.tokens import render_token_grid
from cjm_fasthtml_token_selector.helpers.tokenizer import tokenize
from cjm_fasthtml_token_selector.core.models import TokenSelectorState

# HTML IDs (page-specific)
from cjm_transcript_segmentation.html_ids import SegmentationHtmlIds

# Local imports
from cjm_transcript_segmentation.models import TextSegment
from cjm_transcript_segmentation.components.card_stack_config import (
    SEG_TS_CONFIG, SEG_TS_IDS,
)

## Card Metadata

Left side of the card showing index and timestamp.

In [ ]:
#| export
def _render_card_metadata(
    segment:TextSegment,  # Segment to render metadata for
) -> Any:  # Metadata component
    """Render the left metadata column of a segment card."""
    return Div(
        # Index number
        Span(
            f"#{segment.index + 1}",
            cls=combine_classes(font_size.xs, font_family.mono, font_weight.bold)
        ),
        cls=combine_classes(
            w(16), shrink(0),
            flex_display, flex_direction.col, gap(2),
            p.t(1), opacity(50)
        )
    )

## View Mode Content

Standard text display when card is not in split mode.

In [ ]:
#| export
def _render_view_mode_content(
    segment: TextSegment,  # Segment to render
    card_role: CardRole,  # Role of this card in viewport
    enter_split_url: str,  # URL to enter split mode
) -> Any:  # View mode content component
    """Render the text content in view mode."""
    is_focused = card_role == "focused"
    
    # Text styling based on card role
    text_cls = text_dui.base_content if is_focused else text_dui.base_content.opacity(80)
    
    # For context cards, render simple non-interactive content
    if card_role == "context":
        return Div(
            P(
                segment.text,
                cls=combine_classes(font_size.lg, leading.relaxed, select.none, text_cls)
            ),
            cls=combine_classes(p(1), m.l(-1))
        )
    
    # For focused cards, render interactive content with click handler
    return Div(
        P(
            segment.text,
            cls=combine_classes(font_size.lg, leading.relaxed, select.none, text_cls)
        ),
        # Click hint (visible on hover)
        Div(
            "Click to Split",
            cls=combine_classes(
                font_size.xs, font_weight.bold, text_dui.primary,
                uppercase, opacity(0), opacity(100).group("hover/text"),
                transition.opacity, m.t(1)
            )
        ),
        cls=combine_classes(
            "group/text", cursor.text, border_radius.field, p(1), m.l(-1),
            bg_dui.base_200.hover, transition.colors
        ),
        hx_post=enter_split_url,
        hx_vals=json.dumps({"segment_index": segment.index}),
        hx_swap="none",  # Only OOB swaps, no primary swap
        onclick="if(window.kbNav)window.kbNav.enterMode('split')",
    )

## Split Mode Content

Interactive token grid with caret indicator for splitting.
Uses the `cjm-fasthtml-token-selector` library for token rendering and navigation.

In [ ]:
#| export
def _render_split_mode_content(
    segment:TextSegment,  # Segment to render
    caret_position:int,  # Current caret position (token index)
    split_url:str,  # URL to execute split
    exit_split_url:str,  # URL to exit split mode
) -> Any:  # Split mode content component
    """Render the interactive token display in split mode."""
    tokens = tokenize(segment.text)
    state = TokenSelectorState(
        anchor=caret_position, focus=caret_position, word_count=len(tokens),
    )
    grid = render_token_grid(tokens, SEG_TS_CONFIG, SEG_TS_IDS, state)

    return Div(
        # Token grid from library
        grid,

        # Action buttons
        Div(
            Button(
                lucide_icon("scissors", size=4, cls=str(m.r(2))),
                "Split Here",
                cls=combine_classes(btn, btn_colors.primary, btn_sizes.sm, w(40)),
                hx_post=split_url,
                hx_vals=json.dumps({"segment_index": segment.index}),
                hx_include=f"#{SEG_TS_IDS.anchor_input}",
                hx_swap="none",
                onclick="if(window.kbNav)window.kbNav.exitMode()",
            ),
            Button(
                "Cancel",
                cls=combine_classes(btn, btn_styles.ghost, btn_sizes.sm),
                hx_post=exit_split_url,
                hx_swap="none",
                onclick="if(window.kbNav)window.kbNav.exitMode()",
            ),
            Div(
                "Press ",
                Span("Enter", cls=combine_classes(kbd, kbd_sizes.xs)),
                cls=combine_classes(font_size.xs, text_dui.base_content.opacity(50), m.l.auto)
            ),
            id=SegmentationHtmlIds.SPLIT_MODE_ACTIONS,
            cls=combine_classes(
                flex_display, items.center, gap(2),
                border_dui.base_200, border.t(), p.t(3)
            )
        ),
        id=SegmentationHtmlIds.SPLIT_MODE_CONTAINER
    )

## Card Actions

Hover-visible action buttons for merge and split.

In [ ]:
#| export
def _render_card_actions(
    segment: TextSegment,  # Segment this card represents
    merge_url: str,  # URL for merge action
    enter_split_url: str,  # URL to enter split mode
    show_merge: bool = True,  # Whether to show merge button (first card can't merge)
) -> Any:  # Card actions component
    """Render hover-visible action buttons."""
    return Div(
        # Merge up button
        Button(
            lucide_icon("merge", size=4, cls=str(text_dui.base_content.opacity(60))),
            cls=combine_classes(
                btn, btn_styles.ghost, btn_sizes.xs, btn_modifiers.square,
                opacity(0), opacity(100).group("hover"), opacity(100).focus,
                transition.opacity, tooltip, tooltip_placement.left
            ),
            data_tip="Merge with previous",
            hx_post=merge_url,
            hx_vals=json.dumps({"segment_index": segment.index}),
            hx_swap="none"  # Only OOB swaps, no primary swap
        ) if show_merge else None,
        
        # Split button
        Button(
            lucide_icon("scissors", size=4, cls=str(text_dui.base_content.opacity(60))),
            cls=combine_classes(
                btn, btn_styles.ghost, btn_sizes.xs, btn_modifiers.square,
                opacity(0), opacity(100).group("hover"), opacity(100).focus,
                transition.opacity, tooltip, tooltip_placement.left
            ),
            data_tip="Split segment",
            hx_post=enter_split_url,
            hx_vals=json.dumps({"segment_index": segment.index}),
            hx_swap="none",  # Only OOB swaps, no primary swap
            onclick="if(window.kbNav)window.kbNav.enterMode('split')",
        ),
        cls=combine_classes(
            position.absolute, right(2), top(2),
            flex_display, flex_direction.col, gap(1)
        )
    )

## Main Card Renderer

In [ ]:
#| export
def render_segment_card(
    segment: TextSegment,  # Segment to render
    card_role: CardRole,  # Role of this card in viewport ("focused" or "context")
    is_split_mode: bool,  # Whether this card is in split mode
    caret_position: int,  # Caret position for split mode (word index)
    split_url: str,  # URL to execute split
    merge_url: str,  # URL to merge with previous
    enter_split_url: str,  # URL to enter split mode
    exit_split_url: str,  # URL to exit split mode
    is_first_segment: bool = False,  # Whether this is the first segment (can't merge)
    has_boundary_above: bool = False,  # Source boundary exists above this card
    has_boundary_below: bool = False,  # Source boundary exists below this card
) -> Any:  # Segment card component
    """Render a segment card with view or split mode content."""
    is_focused = card_role == "focused"
    is_context = card_role == "context"
    
    # Content based on mode and role
    if is_split_mode and is_focused:
        content = _render_split_mode_content(
            segment, caret_position, split_url, exit_split_url
        )
    else:
        content = _render_view_mode_content(
            segment, card_role, enter_split_url
        )
    
    # Only show actions on focused cards in view mode
    show_actions = is_focused and not is_split_mode
    
    # Boundary borders only on non-focused cards
    boundary_cls = ""
    if is_context:
        if has_boundary_above:
            boundary_cls = combine_classes(border.t(4), border_dui.neutral)
        if has_boundary_below:
            boundary_cls = combine_classes(boundary_cls, border.b(4), border_dui.neutral)
    
    return Div(
        Div(
            # Left: Metadata
            _render_card_metadata(segment),
            
            # Center: Content
            Div(
                content,
                cls=combine_classes(grow(), min_h(6))
            ),
            
            # Right: Actions (only on focused card in view mode)
            _render_card_actions(
                segment, merge_url, enter_split_url,
                show_merge=not is_first_segment
            ) if show_actions else None,
            
            cls=combine_classes(
                card_body, p(4),
                flex_display, flex_direction.row, gap(4), items.start,
                position.relative, p.r(12) if show_actions else ""
            )
        ),
        id=SegmentationHtmlIds.segment_card(segment.index),
        cls=combine_classes(
            card, "segment-card", "group",
            bg_dui.base_100, 
            w.full,
            transition.all,
            duration(200),
            boundary_cls,
        ),
        data_segment_index=str(segment.index),
        data_card_role=card_role
    )

## Card Renderer Factory

Creates a card renderer callback that captures split/merge URLs and mode state,
compatible with the card stack viewport's `render_card` parameter.

In [ ]:
#| export
def create_segment_card_renderer(
    split_url: str = "",  # URL to execute split
    merge_url: str = "",  # URL to merge with previous
    enter_split_url: str = "",  # URL to enter split mode
    exit_split_url: str = "",  # URL to exit split mode
    is_split_mode: bool = False,  # Whether split mode is active
    caret_position: int = 0,  # Caret position for split mode (word index)
    source_boundaries: Set[int] = None,  # Indices where source_id changes
) -> Callable:  # Card renderer callback: (item, CardRenderContext) -> FT
    """Create a card renderer callback for segment cards."""
    boundaries = source_boundaries or set()
    
    def _render(
        item: Any,  # TextSegment instance
        context: CardRenderContext,  # Render context from card stack library
    ) -> Any:  # Rendered segment card component
        """Render a segment card for the given item and viewport context."""
        idx = context.index
        return render_segment_card(
            segment=item,
            card_role=context.card_role,
            is_split_mode=is_split_mode and (context.card_role == "focused"),
            caret_position=caret_position,
            split_url=split_url,
            merge_url=merge_url,
            enter_split_url=enter_split_url,
            exit_split_url=exit_split_url,
            is_first_segment=context.is_first,
            has_boundary_above=(idx in boundaries),
            has_boundary_below=((idx + 1) in boundaries),
        )
    return _render

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()